In [11]:
#pip install langchain-openai

In [1]:
import os

In [3]:

os.environ["OPENAI_API_KEY"] = ""

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
#llm = ChatOpenAI(api_key="...")

In [7]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing by providing automated testing tools and frameworks that can be used to write and execute test cases for software applications. This can help streamline the testing process, ensure consistent test coverage, and identify bugs or issues more efficiently. Additionally, Langsmith can provide support for writing test scripts, setting up test environments, and analyzing test results to help improve the overall quality of the software being tested.', response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 15, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-790f59f3-cf02-47c7-be2d-cd95336b7ed7-0')

We can also guide its response with a prompt template. Prompt templates convert raw user input to better input to the LLM.

In [8]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [9]:
#We can now combine these into a simple LLM chain:
chain = prompt | llm 

In [12]:
chain.invoke({"input": "how can langsmith help with testing?"})


AIMessage(content="Langsmith is a powerful tool that can greatly assist in testing various aspects of software development. Here are some ways in which Langsmith can help with testing:\n\n1. **Automated Testing:** Langsmith can be used to automate testing processes, which can help in saving time and effort in executing tests repeatedly. By writing scripts using Langsmith, you can automate various test cases and scenarios.\n\n2. **Data Generation:** Langsmith can be utilized to generate test data for various scenarios, which is crucial for testing the software under different conditions. By using Langsmith to generate random or specific data sets, you can ensure comprehensive test coverage.\n\n3. **Integration Testing:** Langsmith can aid in integration testing by simulating various components of the software system and verifying that they work together correctly. This can be done by writing scripts that test the interactions between different modules or APIs.\n\n4. **Performance Testin

The output of a ChatModel (and therefore, of this chain) is a message. However, it's often much more convenient to work with strings. Let's add a simple output parser to convert the chat message to a string.

In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [14]:
#We can now add this to the previous chain:
chain = prompt | llm | output_parser

In [15]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith is a powerful tool that can assist with testing in several ways:\n\n1. **Automated Testing:** Langsmith can be integrated with testing frameworks such as JUnit, TestNG, or PyTest to automate the testing process. It can generate test cases based on the specifications provided, saving time and effort in writing test scripts manually.\n\n2. **Test Data Generation:** Langsmith can generate realistic and diverse test data to ensure thorough test coverage. It can be configured to create various input scenarios to test the system under different conditions.\n\n3. **Boundary Testing:** Langsmith can help in identifying boundary conditions and edge cases that need to be tested. By generating test cases that cover these scenarios, Langsmith can improve the robustness of the testing process.\n\n4. **Regression Testing:** Langsmith can generate regression test suites to ensure that new code changes do not introduce any bugs or issues in the existing functionality. This can help in maint

Retrieval Chain
To properly answer the original question ("how can langsmith help with testing?"), we need to provide additional context to the LLM. Retrieval is useful when you have too much data to pass to the LLM directly. You can then use a retriever to fetch only the most relevant pieces and pass those in.

In this process, we will look up relevant documents from a Retriever and then pass them into the prompt. A Retriever can be backed by anything - a SQL table, the internet, etc - but in this instance we will populate a vector store and use that as a retriever. 

First, we need to load the data that we want to index. To do this, we will use the WebBaseLoader. This requires installing BeautifulSoup:

In [17]:
#pip install beautifulsoup4

In [18]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

Next, we need to index it into a vectorstore. This requires a few components, namely an embedding model and a vectorstore.

In [19]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

Now, we can use this embedding model to ingest documents into a vectorstore. We will use a simple local vectorstore, FAISS, for simplicity's sake.

First we need to install the required packages for that:

In [21]:
#pip install faiss-cpu

In [22]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

First, let's set up the chain that takes a question and the retrieved documents and generates an answer.

In [23]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [24]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing you to visualize test results.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [25]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [26]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by allowing developers to create datasets, run tests on their LLM applications, upload test cases in bulk, create test cases on the fly, export test cases from application traces, and run custom evaluations (both LLM and heuristic based) to score test results. Additionally, LangSmith provides a comparison view for tracking and diagnosing regressions in test scores across multiple revisions of the application, as well as a playground environment for rapid iteration and experimentation with different prompts and models.


## Conversation Retrieval Chain

So how do we turn this chain into one that can answer follow up questions?

We can still use the create_retrieval_chain function, but we need to change two things:

The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.
The final LLM chain should likewise take the whole history into account

Updating Retrieval

In order to update retrieval, we will create a new chain. This chain will take in the most recent input (input) and the conversation history (chat_history) and use an LLM to generate a search query.

In [27]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [28]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM applications, we suggest having LangSmith tracing enabled

You should see that this returns documents about testing in LangSmith. This is because the LLM generated a new query, combining the chat history with the follow-up question.

Now that we have this new retriever, we can create a new chain to continue the conversation with these retrieved documents in mind.

In [29]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [30]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug wh

# Agent

We've so far created examples of chains - where each step is known ahead of time. The final thing we will create is an agent - where the LLM decides what steps to take.

NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access to two tools:

The retriever we just created. This will let it easily answer questions about LangSmith
A search tool. This will let it easily answer questions that require up-to-date information.
First, let's set up a tool for the retriever we just created:

In [31]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [33]:
#pip install -U langchain-community tavily-python

In [37]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = '....'

In [39]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [40]:
tools = [retriever_tool, search]

Now that we have the tools, we can create an agent to use them. We will go over this pretty quickly - for a deeper dive into what exactly is going on, check out the Agent's Getting Started documentation

Install langchain hub first

In [46]:
#pip install langchainhub

In [45]:
#pip install langchain-openai

In [47]:
#Now we can use it to get a predefined prompt
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [48]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing'}`


Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phas

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in various ways throughout the application development lifecycle. Here are some key ways LangSmith can assist with testing:\n\n1. Prototyping: LangSmith supports quick experimentation between prompts, model types, retrieval strategies, and other parameters to help developers understand how the model is performing during the prototyping phase.\n\n2. Debugging: LangSmith provides clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues when things go wrong.\n\n3. Initial Test Set: Developers can create datasets of inputs and reference outputs to run tests on their LLM applications. LangSmith allows for running custom evaluations to score test results.\n\n4. Comparison View: LangSmith offers a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of an application

In [49]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1714297386, 'localtime': '2024-04-28 2:43'}, 'current': {'last_updated_epoch': 1714296600, 'last_updated': '2024-04-28 02:30', 'temp_c': 11.7, 'temp_f': 53.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 10, 'wind_dir': 'N', 'pressure_mb': 1020.0, 'pressure_in': 30.12, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 86, 'cloud': 25, 'feelslike_c': 10.5, 'feelslike_f': 50.9, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 10.6, 'gust_kph': 17.1}}"}, {'url': 'https://www.weathertab.com/en/c/e/04

{'input': 'what is the weather in SF?',
 'output': 'The current weather in San Francisco is partly cloudy with a temperature of 53.1°F (11.7°C). The wind is blowing from the north at 3.6 km/h, and the humidity is at 86%.'}

In [50]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith LLM application testing services'}`


Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for th

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith is a platform that supports LLM application development, monitoring, and testing. Here are some ways LangSmith can help test your LLM applications:\n\n1. Prototyping: LangSmith allows for quick experimentation between prompts, model types, and retrieval strategies to understand how the model is performing during the prototyping phase.\n\n2. Debugging: LangSmith provides tracing capabilities to help debug issues in LLM applications by giving clear visibility and debugging information at each step of an LLM sequence.\n\n3. Initial Test Set: Developers can create datasets of inputs and reference outputs to run tests on their LLM applications and evaluate the results.\n\n4. Comparison View: LangSmith offers a comparison view to track and diagnose regressions in test scores across multiple revisions of your application.\n\n5. Pl

# Serving with LangServe
Now that we've built an application, we need to serve it. That's where LangServe comes in. LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.

While the first part of this guide was intended to be run in a Jupyter Notebook, we will now move out of that. We will be creating a Python file and then interacting with it from the command line.

Install with: